In [16]:
import json
import datetime as dt

In [ ]:
# load expenses.json
with open("local_dev/expenses.json", "r") as f:
    expenses = json.load(f)

expenses['months'].sort(key=lambda x: dt.datetime.strptime(x['month'], '%B %Y'))

In [17]:
dt.datetime.strptime('April 2023', '%B %Y')

datetime.datetime(2023, 4, 1, 0, 0)